In [3]:
# Map the dropdown value to enable_TVDL_strategy
if TT.value == '3':
    enable_TVDL_strategy = 1
elif TT.value == '1':
    enable_TVDL_strategy = 0
elif TT.value == '5':
    enable_TVDL_strategy = 2

def find_matching_result(df, suggestion):
    if suggestion:
        # Create the suggestion string in the same format as the formulation strings
        suggestion_str = f'The formulation is Powderkg = {suggestion["powderkg"]}, wc = {suggestion["wc"]}, materials = {suggestion["materials"]}, curing = {suggestion["curing"]}'
        TrainingDat = f'Powderkg = {suggestion["powderkg"]}, wc = {suggestion["wc"]}, materials = {suggestion["materials"]}, curing = {suggestion["curing"]}'
        
        # Look for a match in the DataFrame
        match = df[df["Formulation"].str.lower() == suggestion_str.lower()]
        
        # If a match was found, return the lab result and TrainingDat
        if not match.empty:
            return match.iloc[0]["Strength"], TrainingDat
        
        # If no match was found, print the suggestion string for debugging
        else:
            print("No match found for suggestion string: ", suggestion_str)

    # If no suggestion provided or no match found, return None
    return None, None


import re
import json

def parse_solution(response):
    # Initialize a dictionary to hold the solution
    solution = {}

    # Function to normalize key names
    def normalize_key(key):
        # Normalize common variations to a standard form
        key_map = {
            'powderkg': 'powderkg',
            'wc': 'wc',
            'materials': 'materials',
            'curing': 'curing'
        }
        for known_key, normalized_key in key_map.items():
            if known_key in key.lower().replace(" ", ""):
                return normalized_key
        return None

    # Try to parse the response as JSON
    try:
        json_data = json.loads(response)
        for key, value in json_data.items():
            normalized_key = normalize_key(key)
            if normalized_key:
                solution[normalized_key] = str(value)
        if solution:  # If we successfully extracted data
            return solution
    except json.JSONDecodeError:
        # If JSON parsing fails, proceed with regex parsing for the plain text format
        keys = ['powderkg', 'wc', 'materials', 'curing']  # Updated to include 'curing'
        for key in keys:
            if key != 'curing':  # For 'curing', we might need a different approach
                match = re.search(fr"{key} = (.*?)(,|$)", response, re.IGNORECASE)
                if match:
                    value = match.group(1).strip()
                    solution[key] = value
                else:
                    # If any key wasn't found using regex, return None
                    return None
            else:
                # Handle 'curing' specifically based on the presence of keywords
                if "ambient" in response.lower():
                    solution["curing"] = "Ambient curing"
                elif "heat" in response.lower():
                    solution["curing"] = "Heat curing"
                else:
                    # If 'curing' condition is not met, return None
                    return None
        return solution  # Return the solution dictionary if all keys were found with regex

    # Return None if neither JSON nor regex parsing succeeded
    return None

    
def format_response_to_model(lab_result):
    """
    Given a lab result, format a response message to the model.
    """
    return f"We've achieved a compressive strength of {lab_result['fc_28d_Lab_validation']} MPa. Let's try to do better!"

def parse_materials(materials_str):
    match = re.search(r'(\d+)/(\d+) FA/GGBFS', materials_str)
    if match:
        return int(match.group(1)) / (int(match.group(1)) + int(match.group(2)))
    else:
        return None
    
def parse_curing(materials_str):
    if "Ambient curing" in materials_str:
        return "ambient"
    elif "Heat curing" in materials_str:
        return "oven"
    else:
        return None

def load_data(csv_path):
    df = pd.read_csv(csv_path)
    df['FA_GGBFS_ratio'] = df['Materials'].apply(parse_materials)
    df['curing'] = df['Materials'].apply(parse_curing)  # Add this line
    return df

df = load_data('Data/DiscoveryData_Sample.csv')

# Initialize empty DataFrame
formulation_df = pd.DataFrame(columns=["Formulation", "Strength"])

# Loop through each row in the original data
for idx, row in df.iterrows():
    
    # Get necessary attributes from row
    powder = row["Powderkg"]
    wc = row["WC"]
    materials = row["Materials"]

    # Extract Fly Ash/GGBFS ratio
    fa_ggbfs = materials.split(",")[0].split("-")[1]
    
    # Extract curing method
    curing_method = materials.split(",")[-1].strip()

    # Remove unwanted string from curing method
    curing_method = curing_method.replace(" (Rao et al. 2018)", "")
    curing_method = curing_method.replace(" (Rao et al.)", "")
    
    # Compressive strength
    strength = row["fc_28dGroundTruth"]
    
    # Create formulation string in the same format as the model's output
    formulation = f'The formulation is Powderkg = {powder}, wc = {wc}, materials = {fa_ggbfs}, curing = {curing_method}'
    
    # Append the formulation and its respective strength to the new DataFrame
    new_row = pd.DataFrame({"Formulation": [formulation], "Strength": [strength]})
    formulation_df = pd.concat([formulation_df, new_row], ignore_index=True)


def handle_openai_error(exception):
    if isinstance(exception, openai.error.RateLimitError):
        print(f"Rate limit error. Will retry after {exception.wait_seconds} seconds.")
        time.sleep(exception.wait_seconds)
    elif isinstance(exception, openai.error.InvalidRequestError):
        print(f"Invalid request: {str(exception)}")
    elif isinstance(exception, openai.error.AuthenticationError):
        print(f"Authentication error: {str(exception)}")
    elif isinstance(exception, openai.error.ServiceUnavailableError):
        print(f"Service unavailable error. Retrying after a delay...")
        time.sleep(5)  # Sleep for 5 seconds before retrying
    elif isinstance(exception, openai.error.APIError):
        print(f"API error: {str(exception)}. Retrying after a delay...")
        time.sleep(5)  # Sleep for 5 seconds before retrying
    elif isinstance(exception, openai.error.Timeout):
        print(f"Timeout error: {str(exception)}. Retrying after a longer delay...")
        time.sleep(10)  # Sleep for 10 seconds before retrying
    else:
        raise exception
        
# -> here we also set the API parameters, such as temperature, etc.

def call_openai_api(messages,temp, max_retries=5, delay=5):
    for i in range(max_retries):
        try:
            response = openai.chat.completions.create(
                model= model_dropdown.value,
                temperature=temp,
                messages=messages,
                max_tokens=500,
                n=1
            )
            return response
        except openai.error.OpenAIError as e:
            
            handle_openai_error(e)
            if i < max_retries - 1:  # i is zero indexed
                time.sleep(delay)  # wait before trying again
                continue
            else:
                raise
                
# Load the text from the file
if  prompt_dropdown.value == 'None 0':
    with open('Prompts/prompts_ID_none 0.txt', 'r') as f:
        lines = f.read().splitlines()
elif prompt_dropdown.value == 'Generic 0':
    with open('Prompts/prompts_ID_generic 0.txt', 'r') as f:
        lines = f.read().splitlines()
elif prompt_dropdown.value == 'Specific 0':
    with open('Prompts/prompts_ID_specific 0.txt', 'r') as f:
        lines = f.read().splitlines()
elif  prompt_dropdown.value == 'None 1':
    with open('Prompts/prompts_ID_none 1.txt', 'r') as f:
        lines = f.read().splitlines()
elif prompt_dropdown.value == 'Generic 1':
    with open('Prompts/prompts_ID_generic 1.txt', 'r') as f:
        lines = f.read().splitlines()
elif prompt_dropdown.value == 'Specific 1':
    with open('Prompts/prompts_ID_specific 1.txt', 'r') as f:
        lines = f.read().splitlines()
elif  prompt_dropdown.value == 'None 2':
    with open('Prompts/prompts_ID_none 2.txt', 'r') as f:
        lines = f.read().splitlines()
elif prompt_dropdown.value == 'Generic 2':
    with open('Prompts/prompts_ID_generic 2.txt', 'r') as f:
        lines = f.read().splitlines()
elif prompt_dropdown.value == 'Specific 2':
    with open('Prompts/prompts_ID_specific 2.txt', 'r') as f:
        lines = f.read().splitlines()
  
# Store the contents in separate variables
#instructions_text = lines[0]
DA_role_text = lines[0]
context_text = lines[1]
iterate_text = lines[2]
VM_role_text = lines[3]


  
# Create the widgets with the loaded text
layout = widgets.Layout(width='auto', height='200px')  # adjust the height and width as needed
#instructions_prompt = widgets.Textarea(value=instructions_text, description='Instructions:', layout=layout)
DA_role_prompt = widgets.Textarea(value=DA_role_text, description='DA Instuctions:', layout=layout)
VM_role_prompt = widgets.Textarea(value=VM_role_text, description='VM Instuctions:', layout=layout)
context_prompt = widgets.Textarea(value=context_text, description='Design Knowledge:', layout=layout)

iterate_prompt = widgets.Textarea(value=iterate_text, description='User:', layout=layout)


# Run to display the text box widgets and the save button

# Set the 'flex' property for each widget inside main_layout
iterate_prompt.layout.flex = '2'       # Adjust the value as needed

#Instructions_box = widgets.HBox([instructions_prompt],layout=widgets.Layout(width='100%', height='100px'))
Iterate_box = widgets.HBox([iterate_prompt],layout=widgets.Layout(width='100%', height='100px'))

title = widgets.HTML("<h2>Main Prompts</h2>")
Prompts = widgets.VBox([title ,DA_role_prompt, VM_role_prompt, Iterate_box, context_prompt])

temperatures_str = model_temperatures.value
# Split the string using commas as a delimiter and convert to float numbers
temperatures = [float(t) for t in temperatures_str.split(',')]
budget = num_development.value
NrOfExper = num_experiments.value

desired_strength = formulation_df["Strength"].quantile(targ_quant.value/100)

num_entries_above_desired = (formulation_df["Strength"] >= desired_strength).sum()

# Print the result

print('SUMMARY\n' +'The design target is to achieve a strength of ',desired_strength, 'MPa within ',num_development.value,' development cycles.\n' +
      'The Experiment is repeated ',num_experiments.value,' times using the ',model_dropdown.value, ' model and the prompt strategy: ',prompt_dropdown.value,'.')
print("There are ", num_entries_above_desired,' formulations above or equal to desired_strength.')


def extract_formulations_from_training_data(training_data):
    pattern = re.compile(r'Powderkg\s*=\s*(\d+),\s*wc\s*=\s*(\d+\.\d+),\s*materials\s*=\s*(\d+\.\d+/\d+\.\d+),\s*curing\s*=\s*(\w+)', re.IGNORECASE)
    training_formulations = [match.group(0) for data in training_data for match in [pattern.search(data)] if match]
    return training_formulations

NameError: name 'pd' is not defined